In [ ]:
import requests
import pandas
from dateutil import parser
host = 'http://18.188.56.207:9200/'
requests.get(host + '_cat/indices/enron').content

b'yellow open enron lVq0is2BTCmgDk2kFyZHTQ 1 1 251735 115558 895.3mb 895.3mb\n'

In [ ]:
doc = {
    "query" : {
        "match_all" : {}
    }
}
import json
r=requests.get(host + 'enron/_search', data=json.dumps(doc), headers={'Content-Type':'application/json'})
print(len(r.json()['hits']['hits']))

10


In [ ]:
def elasticsearch_results_to_df(results):
    '''
    A function that will take the results of a requests.get
    call to Elasticsearch and return a pandas.DataFrame object
    with the results
    '''
    hits = results.json()['hits']['hits']
    data = pandas.DataFrame([i['_source'] for i in hits], index = [i['_id'] for i in hits])
    data['date'] = data['date'].apply(parser.parse)
    return(data)

def print_df_row(row):
    '''
    A function that will take a row of the data frame and print it out
    '''
    print('____________________')
    print('RE: %s' % row.get('subject',''))
    print('At: %s' % row.get('date',''))
    print('From: %s' % row.get('sender',''))
    print('To: %s' % row.get('recipients',''))
    print('CC: %s' % row.get('cc',''))
    print('BCC: %s' % row.get('bcc',''))
    print('Body:\n%s' % row.get('text',''))
    print('____________________')


In [ ]:
# Put elasticsearch results into a pandas.DataFrame object
# df = elasticsearch_results_to_df(r)
# print(df)
# print_df_row(df.iloc[0])

TypeError: Parser must be a string or character stream, not float

In [ ]:
# Query For a full text match in the "text" field
# Uses the "match" query: https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html
doc = {
    "query": {
        "match" : {
            "text" : "important reporting"
        }
    },
    "from" : 0, # Starting message to return.
    "size" : 2000, # Return this many messages. Can't be more than 10,000
}
r=requests.get(host + 'enron/_search',
               data=json.dumps(doc), headers={'Content-Type':'application/json'})
r.raise_for_status()
print("Found %s messages matching the query, of " % r.json()['hits']['total'])
df = elasticsearch_results_to_df(r)
print("Returned %s messages" % df.shape[0])
print_df_row(df.iloc[0])

Found {'value': 10000, 'relation': 'gte'} messages matching the query, of 
Returned 2000 messages
____________________
RE: RE: Reporting replication issue is now fixed
At: 2001-10-09 17:11:18+00:00
From: lynn.blair@enron.com
To: jennifer.lowry@enron.com  group.dl-ets@enron.com
CC: nan
BCC: nan
Body:
	Jennifer, how long has this been a problem?  Is there a concern we have caused
	customers problems in nominating due to bad information?  Thanks. Lynn

 -----Original Message-----
From: 	Lowry, Jennifer   
Sent:	Tuesday, October 09, 2001 10:11 AM
To:	DL-ETS TMS Modification Group
Subject:	Reporting replication issue is now fixed


Yesterday we noticed a problem where reports were not reporting on the correct cycle, or were not picking up information between cycles.  As it turns out, an important table was not being replicated from the application database to the reporting database.  

I was told that this problem has been fixed, and on first inspection of the tables, everything looks corre

In [ ]:
# Query For a text match in the "text" or "subject" fields. Uses the multi-match query:
# https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-multi-match-query.html
doc = {
  "query": {
    "multi_match" : {
      "query":    "settlement",
      "fields": [ "subject", "text" ]
    }
  }
}
r=requests.get(host + 'enron/_search',
               data=json.dumps(doc), headers={'Content-Type':'application/json'})
r.raise_for_status()
print("Found %s messages matching the query, of " % r.json()['hits']['total'])
df = elasticsearch_results_to_df(r)
print("Returned %s messages" % df.shape[0])
print_df_row(df.iloc[9])

Found {'value': 4102, 'relation': 'eq'} messages matching the query, of 
Returned 10 messages
____________________
RE: Status of final statements and replacement invoices
At: 2001-10-24 01:34:50+00:00
From: thailu@ercot.com
To: jackson.amie@enron.com  pratka.amy@enron.com  mitrey.andy@enron.com  williams.angela@enron.com  garza.beth@enron.com  palmer.bill@enron.com  cooper.bob@enron.com  edwards.brady@enron.com  green.brenda@enron.com  smith.carl@enron.com  smith.carl@enron.com  carey.dan@enron.com  sarti.dan@enron.com  leger.dana@enron.com  wessels.david@enron.com  pawlik.debbie@enron.com  bailey.debra@enron.com  dyc.dennis@enron.com  slover.eric@enron.com  nitschmann.frances@enron.com  herndon.gary@enron.com  geissler.ginger@enron.com  striedel.james@enron.com  holland.janet@enron.com  jeffrey.miller@enron.com  doyas.jenny@enron.com  burt.jerry@enron.com  barker.joe@enron.com  favalora.joe@enron.com  forney.john@enron.com  fitzmaurice.kathy@enron.com  minear.kelly@enron.com  koliba.k

In [ ]:
# "OR" query for two phrase matches. Generally you get fancy query parsing with this:
# https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-query-string-query.html
doc = {
    "query": {
        "query_string" : {
            "default_field" : "text",
            "query" : "(reach a settlement) OR (continue to pursue)"
        }
    }
}
r=requests.get(host + 'enron/_search',
               data=json.dumps(doc), headers={'Content-Type':'application/json'})
r.raise_for_status()
print("Found %s messages matching the query" % r.json()['hits']['total']['value'])
df = elasticsearch_results_to_df(r)
print("Returned %s messages" % df.shape[0])
print_df_row(df.iloc[0])

Found 10000 messages matching the query
Returned 10 messages
____________________
RE: Settlement Conversation Recap
At: 2002-05-02 18:26:52+00:00
From: michael.bridges@enron.com
To: koikosp@talgov.com
CC: chris.germany@enron.com
BCC: chris.germany@enron.com
Body:
Hi Pete,

Wanted to recap our conversation yesterday, update you on our timing and get you my contact information.

Regarding a possible settlement, it appears to me that this is a concept that you have considered and are willing to pursue.  We will have a valuation of the contract for you Monday.  It is my suggestion that you review the proposal, make sure that you agree with the outstanding transactions and complete your own valuation.  Once we have agreed on the universe of transactions, you and I will discuss and finalize a settlement amount that we are comfortable will be approved by the creditor committee and bankruptcy judge.

From this point, Enron will begin the process of filing a motion for settlement and schedule a

In [ ]:
df.iloc[9]

,95720
sender,w..cantrell@enron.com
recipients,d..steffes@enron.com leslie.lawner@enron.com ...
cc,NaN
text,\nNGI's Daily Gas Price Index \npublished : Au...
bcc,NaN
date,2001-08-31 14:44:25+00:00
subject,NGI Article: ALJ Advocates Money Settlement i...


In [ ]:
# Do a count of all documents in the database by month
doc = {
    "aggs" : {
        "aggregation_var_name" : {
            "date_histogram" : {
                "field" : "date",
                "interval" : "month"
            }
        }
    }
}
r=requests.get(host + 'enron/_search',
               data=json.dumps(doc), headers={'Content-Type':'application/json'})
r.raise_for_status()
def aggregation_to_df(response,var_name='aggregation_var_name'):
    r = response.json()
    r['aggregations'][var_name]
    df = pandas.DataFrame(r['aggregations'][var_name]['buckets'])
    df['date'] = df['key_as_string'].apply(parser.parse)
    df = df[(df['date'] >= '1999-01-1') & (df['date'] < '2002-07-01')]
    df = df[['date','doc_count']]
    return df
df = aggregation_to_df(r)
print(df)

                         date  doc_count
228 1999-01-01 00:00:00+00:00         65
229 1999-02-01 00:00:00+00:00         43
230 1999-03-01 00:00:00+00:00         50
231 1999-04-01 00:00:00+00:00         45
232 1999-05-01 00:00:00+00:00        338
233 1999-06-01 00:00:00+00:00        327
234 1999-07-01 00:00:00+00:00        446
235 1999-08-01 00:00:00+00:00        509
236 1999-09-01 00:00:00+00:00        588
237 1999-10-01 00:00:00+00:00        643
238 1999-11-01 00:00:00+00:00        594
239 1999-12-01 00:00:00+00:00       1248
240 2000-01-01 00:00:00+00:00       2142
241 2000-02-01 00:00:00+00:00       2471
242 2000-03-01 00:00:00+00:00       3001
243 2000-04-01 00:00:00+00:00       2991
244 2000-05-01 00:00:00+00:00       3718
245 2000-06-01 00:00:00+00:00       4820
246 2000-07-01 00:00:00+00:00       4493
247 2000-08-01 00:00:00+00:00       6200
248 2000-09-01 00:00:00+00:00       6872
249 2000-10-01 00:00:00+00:00       8273
250 2000-11-01 00:00:00+00:00      10617
251 2000-12-01 0

In [ ]:
# Do a count of all documents matching a query by month
doc = {
    "query": {
        "match" : {
            "text" : "important reporting"
        }
    },
    "aggs" : {
        "aggregation_var_name" : {
            "date_histogram" : {
                "field" : "date",
                "interval" : "month"
            }
        }
    }
}
r=requests.get(host + 'enron/_search',
               data=json.dumps(doc), headers={'Content-Type':'application/json'})
r.raise_for_status()
df = aggregation_to_df(r)
print(df)

                         date  doc_count
228 1999-01-01 00:00:00+00:00         12
229 1999-02-01 00:00:00+00:00          7
230 1999-03-01 00:00:00+00:00          2
231 1999-04-01 00:00:00+00:00          8
232 1999-05-01 00:00:00+00:00         12
233 1999-06-01 00:00:00+00:00         13
234 1999-07-01 00:00:00+00:00         21
235 1999-08-01 00:00:00+00:00         22
236 1999-09-01 00:00:00+00:00         14
237 1999-10-01 00:00:00+00:00         15
238 1999-11-01 00:00:00+00:00         31
239 1999-12-01 00:00:00+00:00         55
240 2000-01-01 00:00:00+00:00         89
241 2000-02-01 00:00:00+00:00        104
242 2000-03-01 00:00:00+00:00        116
243 2000-04-01 00:00:00+00:00         92
244 2000-05-01 00:00:00+00:00        157
245 2000-06-01 00:00:00+00:00        176
246 2000-07-01 00:00:00+00:00        180
247 2000-08-01 00:00:00+00:00        268
248 2000-09-01 00:00:00+00:00        312
249 2000-10-01 00:00:00+00:00        403
250 2000-11-01 00:00:00+00:00        625
251 2000-12-01 0

In [ ]:
# Search an exact match in a specific feild
doc = {
    "query": {
        "match" : {
            "recipients" : "stephen.schwarzbach@enron.com"
        }
    },
}
r=requests.get(host + 'enron/_search',
               data=json.dumps(doc), headers={'Content-Type':'application/json'})
r.raise_for_status()
print("Found %s messages matching the query, of " % r.json()['hits']['total']['value'])
df = elasticsearch_results_to_df(r)
print("Returned %s messages" % df.shape[0])
print_df_row(df.iloc[0])

Found 16 messages matching the query, of 
Returned 10 messages
____________________
RE: RE: REQUEST - 2001 Operating Expenses - High Priority
At: 2001-10-02 22:52:12+00:00
From: tracy.geaccone@enron.com
To: stephen.schwarzbach@enron.com
CC: nan
BCC: nan
Body:
No PGG?

 -----Original Message-----
From: 	Schwarzbach, Stephen  
Sent:	Tuesday, October 02, 2001 9:51 AM
To:	Geaccone, Tracy; Tran, Agatha; Killen, Faith; Lee, Dana; Smith, Stephanie; Davis, Rachel A.; Vargas, Hope; Evans, Stephen; Sparger, Bob; Smith, Jeff E.; Myers, Thomas; Ueckert, Allen W.; Copeland, Erin; Selzer, Howard; Boyd, Mike; Smith, Kirk; Roper, Kerry; Galvan, Michael S.; Wood, Stephen; West, Terry
Cc:	Lindsey, Mark E.
Subject:	REQUEST - 2001 Operating Expenses - High Priority

Hello everyone.  I know that most, if not all, of you are busy with quarter close, 2002 Plan, and possibly Board of Directors presentations and that this is not the ideal time to have more added to your plate.  However, in conjunction to with 

# start of Lei Xue Part

In [ ]:
# 那个CFO 的email list 看看？ 竟然才7条，可恶！！
doc = {
    "query": {
        "multi_match": {
            "query": "andrew.fastow@enron.com",
            "fields": ["to", "cc", "bcc", "recipients"]
        }
    }
}
r=requests.get(host + 'enron/_search',
               data=json.dumps(doc), headers={'Content-Type':'application/json'})
r.raise_for_status()
print("Found %s messages matching the query, of " % r.json()['hits']['total']['value'])
df = elasticsearch_results_to_df(r)
print("Returned %s messages" % df.shape[0])
print_df_row(df.iloc[0])


Found 7 messages matching the query, of 
Returned 7 messages
____________________
RE: RE: The National Forum on Corporate Finance
At: 2001-02-26 11:20:00+00:00
From: vince.kaminski@enron.com
To: andrew.fastow@enron.com
CC: daveike@rice.edu  vince.kaminski@enron.com
BCC: daveike@rice.edu  vince.kaminski@enron.com
Body:
Andy,

 Thanks. I shall forward  your message to 
Prof. Ikenberry.

Vince




From: Andrew S Fastow/ENRON@enronXgate on 02/22/2001 01:45 PM
To: Vince J Kaminski/HOU/ECT@ECT
cc:  
Subject: RE: The National Forum on Corporate Finance

Vince:

I would be interested in participating.  Thanks.

Andy

 -----Original Message-----
From:  Kaminski, Vince  
Sent: Monday, February 05, 2001 10:23 AM
To: Andrew S Fastow/HOU/ECT@ENRON
Cc: Kaminski, Vince
Subject: The National Forum on Corporate Finance

Andy,

I am sending you a draft oof a proposal regarding national forum for top 
finance practitioners and
academics. The idea came from a professor at Rice University who
has already r

In [ ]:
mapping = requests.get(host + 'enron/_mapping').json()
print(mapping)

{'enron': {'mappings': {'properties': {'bcc': {'type': 'keyword'}, 'body': {'type': 'text', 'analyzer': 'english'}, 'cc': {'type': 'keyword'}, 'date': {'type': 'date'}, 'recipients': {'type': 'keyword'}, 'scroll': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'scroll_id': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}, 'sender': {'type': 'keyword'}, 'subject': {'type': 'text', 'analyzer': 'english'}, 'text': {'type': 'text', 'fields': {'keyword': {'type': 'keyword', 'ignore_above': 256}}}}}}}


In [ ]:
# 关键词列表查找，发现了10000多条，可恶，我然后加入了时间搜查, 00-02的结果还是有1万+条
keywords = [
    "Revenue Recognition", "Special Purpose Entity", "Off-Balance-Sheet",
    "Derivatives", "Mark-to-Market Accounting", "Internal Audit",
    "Compliance", "Risk Management", "Code of Ethics",
    "Accounting Treatment", "Financial Statements", "Asset Restructuring",
    "Capital Transactions", "SEC", "Investigation", "Litigation",
    "Compliance Review", "Executive Meeting", "Board of Directors",
    "Financial Reporting", "Disclosure"
]

doc = {
    "query": {
        "bool": {
            "must": [
                {
                    "query_string": {
                        "query": " OR ".join(keywords),
                        "fields": ["subject"]
                    }
                },
                {
                    "range": {
                        "date": {
                            "gte": "2000-01-01",
                            "lte": "2002-12-31"
                        }
                    }
                }
            ]
        }
    },
}
r=requests.get(host + 'enron/_search',
               data=json.dumps(doc), headers={'Content-Type':'application/json'})
r.raise_for_status()
print("Found %s messages matching the query, of " % r.json()['hits']['total']['value'])
df = elasticsearch_results_to_df(r)
print("Returned %s messages" % df.shape[0])
print_df_row(df.iloc[0])


HTTPError: 400 Client Error: Bad Request for url: http://18.188.56.207:9200/enron/_search

# End of Lei Xue Part

**Start** of Yi Xia Part

不太确定如何把所有的email放dataframe里面。我现在local的email里面找了一下。
跑下面的code需要先download email。 下载的邮件一共是9万多封
```
# This is formatted as code
```



# Start of Yi Xia Part

In [ ]:
import os
import pandas as pd
from email import policy
from email.parser import BytesParser
from nltk.tokenize import word_tokenize
import nltk

folder_path = r"C:\Users\goxia\OneDrive\!INTA6450DAS\GroupProject\inta6450-emails\dataset"

email_data = []

import json

for filename in os.listdir(folder_path):
    if filename.endswith('.json'):  # assuming the email files are in .json format
        file_path = os.path.join(folder_path, filename)

        # Open and parse the JSON file
        with open(file_path, 'r', encoding='utf-8') as file:
            email = json.load(file)

        # Extract the email text and subject (from JSON structure)
        text = email.get('text', 'No Text')
        subject = email.get('subject', 'No Subject')

        # Add the email's subject and body to the list
        email_data.append({'subject': subject, 'body': text})

df = pd.DataFrame(email_data)
print(df.head())

print(df.iloc[3]['body'])
print(df.iloc[3]['subject'])

df['tokenized_body'] = df['body'].apply(lambda x: word_tokenize(x.lower()))

df['tokenized_body'] = df['tokenized_body'].apply(lambda tokens: list(dict.fromkeys(tokens)))

print(df.head())

print(df.iloc[3]['tokenized_body'])

nltk.download('stopwords')

from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords

corpus = {}

stemmer = PorterStemmer()

stopwords_set = set(stopwords.words('english'))

for i in range(len(df)):
    text = df.iloc[i]['tokenized_body']

    for j in range(len(text)):
        tk = text[j]
        if tk not in corpus.keys():
            corpus[text[j]] = 1
        else:
            corpus[text[j]] += 1

# print(corpus)
key_to_print = 'requirement'

value = corpus.get(key_to_print)

print(len(corpus))

corpus_short = {}

signs = ['requirement','stock', 'price', 'market', 'transaction', 'buy', 'sell', 'shares', 'money', 'short', 'long', 'put']

for i in signs:
    try:
        value = corpus.get(i)
        print(value)
    except:
        print('not found')

# 847
# 2643
# 7369
# 7986
# 3192
# 3893
# 3056
# 1059
# 3761
# 3226
# 5034
# 5339

count = 0
tf = pd.DataFrame()
result = []

for i in range(len(df)):
    k = 0
    tk = df.iloc[i]['tokenized_body']
    for j in tk:
        if j in signs:
            k+=1
        if k>11:
            count+=1
            result.append({'index': i, 'body': df.iloc[i]['body']})
#           print(df.iloc[i]['body'])

tf = pd.DataFrame(result)
print(count)

#14283

count = 0
sf = pd.DataFrame()
result = []

signs2 = ['valuation']


for i in range(len(df)):
    k = 0
    tk = df.iloc[i]['tokenized_body']
#     if k < 5:
#         print(tk)
#     k+=1
    for j in tk:
        if j in signs2:

            result.append({'index': i, 'body': df.iloc[i]['body']})
            count+=1
            continue
#           print(df.iloc[i]['body'])

sf = pd.DataFrame(result)
print(count)

#681

for i in range(10):
    print(sf.iloc[i]['body'])

# Example email found by above method:

# Stinson Gibner
# 06/29/2000 09:55 AM
# To: Vince J Kaminski/HOU/ECT@ECT
# cc:
# Subject: EBS VaR Transaction Policy

# FYI

# ---------------------- Forwarded by Stinson Gibner/HOU/ECT on 06/29/2000
# 09:54 AM ---------------------------


# 	Barry Pearce @ ENRON COMMUNICATIONS
# 	06/29/2000 09:09 AM

# To: Stinson Gibner/HOU/ECT@ECT, Dale Surbey/LON/ECT@ECT, Ted
# Murphy/HOU/ECT@ECT
# cc: Lou Casari/Enron Communications@Enron Communications, John Echols/Enron
# Communications@Enron Communications, Jim Fallon/Enron Communications@Enron
# Communications
# Subject: EBS VaR Transaction Policy

# Hey you guys,

# We are trying to implement a 'VaR transaction' policy - and would like your
# opinion.

# This is going to be tough - because I'm not sure we have implemented a
# similiar policy across any of our other 'books' - that is - we looking to
# bring in all the accrual/operational assets as well as the MTM stuff
# (lambdas). To have a real-live 'configuration' of the system.

# If assets/routes/servers etc...are added - what is the impact on the 'value'
# of the system and what it's worth.

# John has attached a draft below - for your review and thoughts.

# I can see how this works in a trading environment - when you actually know
# the VaR of your whole trading portfolio. However - I've not seen this done
# with a mixture of MTM & accrual assets. Add the spice of a 'operational
# system' valuation  - and this will be tough to quantify and model.

# Step 1 - configure system and value it
# Step 2 - calculate a VaR on this. We will need to do some work here!
# Step 3 - calculate the incremental VaR of new deals/amendements/reconfigs etc
# - tough....

# See what you think?

# B.






# 	John Echols
# 	06/28/00 05:41 PM

# 		 To: Jim Fallon/Enron Communications@Enron Communications, Barry Pearce/Enron
# Communications@Enron Communications, Lou Casari/Enron Communications@Enron
# Communications
# 		 cc:
# 		 Subject: Policies

# Here is a first rough draft of a "value @ risk" transaction policy.

# I would like you to start looking at where we are going on the policy and get
# some early thinking going on limits for the V@R.  For example, should we
# effectively shut down all server relocations without approval,  or allow some
# level of MB of storage to be moved around or reconfigured?

# I need some commercial and industry realism for this.  We may need Rick
# Paiste or your industry helpers (Marquardt, etc. to help us).

# Barry,  Lou,  I need your input.


按照一系列关键词找还是有1万多封。现在打算试一下outlier分析来找。现在还是太多了。

只按照一个关键词找是几百封， 要是这个词找的对好像有点戏能筛出5封。

END of Yi Xia section.

# END of Yi Xia section.

